### This file is to create json file from [crello dataset](https://github.com/CyberAgentAILab/canvas-vae/blob/main/docs/crello-dataset.md).
- We use the crello-dataset-v1 in this work.
- We offer a json creator and a json parser demos below.

In [7]:
import tensorflow as tf
from typing import Any, Dict
from PIL import Image
import io
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Image as ipyImage
from ipycanvas import Canvas
import time
import json
from base64 import b64encode, b64decode

JSON_FILE_PATH = '../data/model_test_input/crello_samples'

In [3]:
dataset = tf.data.Dataset.list_files('../data/crello-dataset-v1/test-*.tfrecord')
dataset = tf.data.TFRecordDataset(dataset)

2022-09-13 16:34:10.345680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def parse(serialized: bytes) -> Dict[str, Any]:
    """Explicitly parse specified fields."""
    context, sequence, _ = tf.io.parse_sequence_example(
        serialized,
        context_features={
            'id': tf.io.FixedLenFeature((), tf.string),
            'group': tf.io.FixedLenFeature((), tf.string),
            'format': tf.io.FixedLenFeature((), tf.string),
            'category': tf.io.FixedLenFeature((), tf.string),
            'canvas_height': tf.io.FixedLenFeature((), tf.int64),
            'canvas_width': tf.io.FixedLenFeature((), tf.int64),
            'length': tf.io.FixedLenFeature((), tf.int64),
        },
        sequence_features={
            'type': tf.io.FixedLenSequenceFeature((), tf.string),
            'left': tf.io.FixedLenSequenceFeature((), tf.float32),
            'top': tf.io.FixedLenSequenceFeature((), tf.float32),
            'width': tf.io.FixedLenSequenceFeature((), tf.float32),
            'height': tf.io.FixedLenSequenceFeature((), tf.float32),
            'color': tf.io.FixedLenSequenceFeature((3,), tf.int64),
            'opacity': tf.io.FixedLenSequenceFeature((), tf.float32),
            'image_bytes': tf.io.FixedLenSequenceFeature((), tf.string),
        })
    return {**context, **sequence}

#### Json Creator

In [8]:
dataset_parse = dataset.map(parse)
index = 0
sampleID = 1
for example in dataset_parse.as_numpy_iterator():
    index += 1
    if index == sampleID:
        elementTypes = [i.decode("utf-8") for i in example['type']]
        canvas = {
            'id': example['id'].decode("utf-8"),
            'group': example['group'].decode("utf-8"),
            'format': example['format'].decode("utf-8"),
            'category': example['category'].decode("utf-8"),
            'canvas_width': int(example['canvas_width']),
            'canvas_height': int(example['canvas_height']),
            'length': int(example['length']),
            'types': elementTypes,
        }
        for i in range(len(example['type'])):
            element = {
                'type': example['type'][i].decode("utf-8"),
                'left': float(example['left'][i]),
                'top': float(example['top'][i]),
                'width': float(example['width'][i]),
                'height': float(example['height'][i]),
                'color': example['color'][i].tolist(),
                'opacity': float(example['opacity'][i]),
                'image_bytes': b64encode(example['image_bytes'][i]).decode("utf-8"),
                'depth': i,
            }
            canvas[f'element_{i}'] = element
        # print(canvas) 
        with open(os.path.join(JSON_FILE_PATH, f'json_sample_{sampleID}.json'), 'w') as outfile:
            json.dump(canvas, outfile)

#### Json parser

In [9]:
# sampleID = 1
with open(os.path.join(JSON_FILE_PATH, f'json_sample_{sampleID}.json')) as json_file:
    data = json.load(json_file)
    canvas_width = data['canvas_width']
    canvas_height = data['canvas_height']
    canvas_image = Canvas(width=canvas_width, height=canvas_height, sync_image_data=True)

    for i in range(data['length']):
    # for i in range(6, 7):
        x = data[f'element_{i}']['left']*canvas_width
        y = data[f'element_{i}']['top']*canvas_height
        width = data[f'element_{i}']['width']*canvas_width
        height = data[f'element_{i}']['height']*canvas_height
        if data[f'element_{i}']['type'] != 'textElement':
            image = ipyImage.from_file(io.BytesIO(b64decode(data[f'element_{i}']['image_bytes'])))
            canvas_image.draw_image(image, x, y, width, height)
            time.sleep(1)
        else:
            # print(i)
            canvas_image.font = f'{height}px sans-serif'
            color_rgb = data[f'element_{i}']['color']
            color_hex = '#%02x%02x%02x' % (color_rgb[0], color_rgb[1], color_rgb[2])
            canvas_image.stroke_style = color_hex
            canvas_image.fill_style = color_hex
            canvas_image.fill_text('TEXT', x, y+height, max_width=width)
            time.sleep(1) # sleep more time when text can not be fully stored
            # canvas_image.stroke_rect(x, y, width, height)
canvas_image

Canvas(height=298, sync_image_data=True, width=419)